In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torchsummary import summary
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset 
import numpy as np
import torch.optim as optim

In [67]:
!pip install efficientnet_pytorch

In [68]:
from efficientnet_pytorch import EfficientNet

In [69]:
resnet152 = models.resnet152(pretrained = True)
resnet101 = models.resnet101(pretrained = True)
MnasNet = models.mnasnet1_0(pretrained = True)
BNet_B5 = EfficientNet.from_pretrained("efficientnet-b5")
models_ = [resnet152, resnet101, MnasNet, BNet_B5]
for model in models_:
  for para in model.parameters():
    para.requires_grad = False

Loaded pretrained weights for efficientnet-b5


In [70]:
for name, child in resnet101.layer4.named_children():
  if name == "2":
    for para in child.parameters():
      para.requires_grad = True
num_feature = resnet101.fc.in_features
resnet101.fc = nn.Linear(num_feature, 42)
for para in resnet101.fc.parameters():
  para.requires_grad = True
summary(resnet101, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [71]:
from imutils import paths
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import random

In [ ]:
def get_data(path):
    image_path = list(paths.list_images(path))
    random.shuffle(image_path)
    label = [p.split(os.path.sep)[-2] for p in image_path]
    list_image = []
    for path in image_path:
      image = load_img(path, target_size = (224,224,3))
      list_image.append(image)
    LE = LabelEncoder()
    Label = LE.fit_transform(label)
    OHE = OneHotEncoder(sparse = False)
    Label = OHE.fit_transform(Label.reshape(-1, 1))
    Data = np.asarray(list_image)
    return Data, Label
Data, Label = get_data("/content/drive/My Drive/Colab datasets/shoppe")
X_train, X_valid, y_train, y_valid = train_test_split(Data, Label, test_size = 0.2)

In [ ]:
class ShopeeData(Dataset):
  def __init__(self, X, y, transforms):
     self.X = X
     self.y = y
     self.transform = transforms
  def __len__(self):
    return len(self.X)
  def __getitem__(self,idx):
    data = self.X[idx]
    data = self.transform(data)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0,5))])
Ds_train = ShopeeData(X_train, y_train, transform)
Ds_test = ShopeeData(X_valid, y_valid, transform)

Dl_train = DataLoader(Ds_train, batch_size = 64, shuffle = True)
Dl_test = DataLoader(Ds_test, batch_size = 64, shuffle = True)

In [ ]:
def training_procedure(model, num_epoch, model_path, weight_path, lr = 0.001):
  Loss = []
  Acc = []
  Epoch = []
  model.to_device(device)
  parameters = filter(lambda x: x.requires_grad, model.parameters())
  optimizer = optim.SGD(parameters, lr = lr)
  for epoch in range(1, num_epoch + 1):
    model.train()
    sum_loss = 0
    total = 0
    correct = 0
    for X, y in Dl_train:
      X.to_device(device)
      y.to_device(device)
      optimizer.zero_grad()
      y_pred = model(X)
      loss = F.cross_entropy(y_pred, y)
      loss.backward()
      optimizer.step()
      sum_loss += loss.item()*(y.size(0))
      total += y.size(0)
      _, predicted = torch.max(y_pred, 1)
      _, label = torch.max(y, 1)
      correct += (predicted == label).sum().item()
    Loss.append(sum_loss/total)
    Acc.append(correct/total)
    Epoch.append(epoch)
    if epoch % 5 == 0:
      print("Loss: {} and Accuracy: {}".format(sum_loss/total, correct/total))
  print("The training process is done")
  torch.save(model, model_path)
  print("model saved")
  torch.save(model.state_dict(), weight_path)
def validation(model):
  correct = 0
  total = 0
  with torch.no_grad():
      model.val()
      for X, y in Dl_test():
        X.to_device(device)
        y.to_device(device)
        y_pred = model(X)
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == label).sum().item()
        total += y.size(0)
    print("Accuracy of the model: {} ", correct/total)